In [3]:
import pandas as pd
import os

In [ ]:
notebook_dir = os.getcwd()

geocoded_csv_path = os.path.join(notebook_dir, '..', 'data', 'geocoded_addresses_final.csv')
geocoded = pd.read_csv(geocoded_csv_path)

checkpoint3_csv_path = os.path.join(notebook_dir, '..', 'data', 'checkpoints', 'checkpoint3_geocoded.csv')
checkpoint3 = pd.read_csv(checkpoint3_csv_path)

In [33]:
checkpoint3['address'] = checkpoint3['Location'].astype(str) + ', Lawrence, MA'

# merge with geocoded data
merged = checkpoint3.merge(
    geocoded.rename(columns={'latitude': 'geo_lat', 'longitude': 'geo_lon'}),
    on='address',
    how='left'
)

# replace latitude/longitude in checkpoint3 with values from geocoded where available
merged['Latitude'] = merged['geo_lat'].combine_first(merged['Latitude'])
merged['Longitude'] = merged['geo_lon'].combine_first(merged['Longitude'])

# drop helper columns
merged = merged.drop(columns=['address', 'geo_lat', 'geo_lon','cleaned_address'])

# save or use `merged` as the updated checkpoint3
checkpoint4 = merged

In [34]:
checkpoint4.head()

,Incident #,Date,Type,Location,Arrested,Location Prefix,Name,DOB,Charges,Latitude,Longitude
0,18015719.0,2018-04-01 00:02:35,ALARM/BURG,14 BROADWAY,No,FELO MOTORS,NaN,NaN,NaN,42.711509,-71.174821
1,18015720.0,2018-04-01 00:03:21,THREATS/PROG,131 SPRUCE ST,No,NaN,NaN,NaN,NaN,42.719151,-71.173651
2,18015721.0,2018-04-01 00:11:06,TOW OF M/V,481 S BROADWAY,No,TRESPASS,NaN,NaN,NaN,42.683771,-71.155965
3,18015722.0,2018-04-01 00:13:45,LOUD NOISE,550 BROADWAY,No,NaN,NaN,NaN,NaN,42.716258,-71.177648
4,18015724.0,2018-04-01 00:23:19,DOMESTIC/PAST,GARDEN ST & NEWBURY ST,No,NaN,NaN,NaN,NaN,42.709178,-71.157889


In [38]:
output_path = os.path.join(notebook_dir, '..', 'data', 'checkpoints', 'checkpoint4_geocoded.csv')
checkpoint4.to_csv(output_path, index=False)

### After Geocoding...

### Finding if Lat and Long are within Massachusetts

In [ ]:
import networkx as nx
import osmnx as ox
import pandas as pd
import os
from shapely.geometry import Point

In [ ]:
notebook_dir = os.getcwd()

checkpoint7_path = os.path.join(notebook_dir, '..', 'data', 'checkpoints', 'checkpoint7_serious_crimes.csv')
checkpoint7_df = pd.read_csv(checkpoint7_path)


In [ ]:
# Get Massachusetts boundary polygon
massachusetts = ox.geocode_to_gdf('Massachusetts, USA')
mass_polygon = massachusetts.geometry.iloc[0]

# Drop rows with missing coordinates
checkpoint7_df = checkpoint7_df.dropna(subset=['Latitude', 'Longitude'])

# Create Point objects for each row
checkpoint7_df['geometry'] = checkpoint7_df.apply(
    lambda row: Point(row['Longitude'], row['Latitude']), axis=1
)

# Filter rows where the point is inside the Massachusetts polygon
checkpoint7_df_filtered = checkpoint7_df[checkpoint7_df['geometry'].apply(lambda point: point.within(mass_polygon))]

# Drop the geometry column if no longer needed
checkpoint7_df_filtered = checkpoint7_df_filtered.drop(columns=['geometry'])

checkpoint7_df_filtered.head()

,Incident #,Date,Type,Location,Arrested,Location Prefix,Name,DOB,Charges,Latitude,Longitude,person_id,category,crime_severity,Year
0,18015719.0,2018-04-01 00:02:35,ALARM/BURG,14 BROADWAY,No,FELO MOTORS,NaN,NaN,NaN,42.711509,-71.174821,NaN,FIRE_AND_ARSON_INCIDENTS,Non-Serious,2018
1,18015720.0,2018-04-01 00:03:21,THREATS/PROG,131 SPRUCE ST,No,NaN,NaN,NaN,NaN,42.719151,-71.173651,NaN,VIOLENT_AND_WEAPON_OFFENSES,Non-Serious,2018
2,18015721.0,2018-04-01 00:11:06,TOW OF M/V,481 S BROADWAY,No,TRESPASS,NaN,NaN,NaN,42.683771,-71.155965,NaN,MOTOR_VEHICLE_INCIDENTS,Non-Serious,2018
3,18015722.0,2018-04-01 00:13:45,LOUD NOISE,550 BROADWAY,No,NaN,NaN,NaN,NaN,42.716258,-71.177648,NaN,PUBLIC_DISTURBANCES,Non-Serious,2018
4,18015724.0,2018-04-01 00:23:19,DOMESTIC/PAST,GARDEN ST & NEWBURY ST,No,NaN,NaN,NaN,NaN,42.709178,-71.157889,NaN,DOMESTIC_DISPUTES_AND_PROTECTION,Serious,2018


In [ ]:
print("Latitude range:", checkpoint7_df_filtered['Latitude'].min(), "to", checkpoint7_df_filtered['Latitude'].max())
print("Longitude range:", checkpoint7_df_filtered['Longitude'].min(), "to", checkpoint7_df_filtered['Longitude'].max())
print()
print("Original rows:", len(checkpoint7_df))
print("Filtered rows (in MA):", len(checkpoint7_df_filtered))

Latitude range: 42.3463578 to 42.826998
Longitude range: -71.479049 to -70.984656

Original rows: 353470
Filtered rows (in MA): 347103


In [ ]:
checkpoint8_path = os.path.join(notebook_dir, '..', 'data', 'checkpoints', 'checkpoint8_mass_filtered.csv')
checkpoint7_df_filtered.to_csv(checkpoint8_path, index=False)
print(f"Filtered data saved to: {checkpoint8_path}")


Filtered data saved to: c:\Users\Indel\Documents\gatewayinitiative-lawrencepd\scripts\..\data\checkpoints\checkpoint8_mass_filtered.csv
